# Portfolio Performance vs. Benchmark Analysis

This notebook performs comprehensive backtesting and performance analysis of algorithmic trading portfolios against benchmarks.
- **Automated Portfolio Detection**: Finds latest portfolio configuration from Excel files
- **Dual Strategy Framework**: Long-term (quarterly) and short-term (weekly) rebalancing strategies
- **Robust Backtesting Engine**: Uses `bt` library with risk-free rate integration and proper rebalancing frequencies
- **Train-Test Validation**: 80/20 split ensuring unbiased out-of-sample performance evaluation
- **Professional Reporting**: QuantStats HTML reports with comprehensive risk-adjusted metrics
- **Benchmark Analysis**: Systematic comparison against market indices with statistical significance testing

## **Environment Setup**

Import essential libraries and configure the analysis environment. The setup includes:
- **Data Processing**: pandas, numpy for quantitative analysis
- **Backtesting**: bt library for systematic strategy testing
- **Performance Analytics**: QuantStats for professional risk-return analysis
- **Utilities**: Custom modules for data loading and filtering

In [9]:
# DataFrame & System Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay
import os, glob, re, warnings
from py.utils import load_and_filter_data

# Import QuantStats and BT libraries
import bt
from py.quantstats_fix import *
qs.extend_pandas()

# Suppress warnings and configure logging
warnings.filterwarnings("ignore")
logging.getLogger('matplotlib.font_manager').disabled = True

## **Data Loading and Configuration**

Automated data pipeline that:
- **Auto-detects** the most recent portfolio configuration file
- **Extracts** portfolio weights, benchmark indices, and risk-free rate
- **Loads** 10-year historical price data for comprehensive analysis
- **Validates** data integrity and alignment across all instruments

In [10]:
# Auto-detect latest portfolio and set analysis period
initial_end_date = (datetime.today() - BDay(1)).to_pydatetime()
expected_file = f'portfolios/portfolio-{datetime.date(initial_end_date)}.xlsx'

if os.path.exists(expected_file):
    end_date, output_file = initial_end_date, expected_file
else:
    portfolio_files = glob.glob('portfolios/portfolio-*.xlsx')
    output_file = max(portfolio_files, key=os.path.getmtime)
    date_match = re.search(r'portfolio-(\d{4}-\d{2}-\d{2})\.xlsx', output_file)
    end_date = pd.to_datetime(date_match.group(1)).to_pydatetime()

start_date = end_date - timedelta(days=10*365)

# Load configurations and extract parameters
sheets = pd.read_excel(output_file, sheet_name=None)
portfolio_long_df, portfolio_short_df = sheets["long_term_portfolio"], sheets["short_term_portfolio"]
benchmark_long, benchmark_short = sheets["benchmark_long_term_portfolio"]['Benchmark'].values[0], sheets["benchmark_short_term_portfolio"]['Benchmark'].values[0]

# Load risk-free rate from 'risk_free' sheet 'Close' column
risk_free_df = sheets["risk_free"].set_index(sheets["risk_free"].columns[0])
risk_free_rate = risk_free_df['Close'].iloc[-1] / 100

# Process weights and extract tickers
for df in [portfolio_long_df, portfolio_short_df]:
    df['Weight'] = df['Weight'].replace('%', '', regex=True).astype(float)

weights_long = portfolio_long_df.set_index('Ticker')['Weight'].to_dict()
weights_short = portfolio_short_df.set_index('Ticker')['Weight'].to_dict()
portfolio_long_tickers, portfolio_short_tickers = list(weights_long.keys()), list(weights_short.keys())

print(f"Analysis: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')} | File: {output_file}")
print(f"Benchmarks: {benchmark_long} (Long), {benchmark_short} (Short) | Risk-free: {risk_free_rate:.4f}")

Analysis: 2015-06-09 to 2025-06-06 | File: portfolios/portfolio-2025-06-06.xlsx
Benchmarks: YYY (Long), XRLV (Short) | Risk-free: 0.0424


In [11]:
# Load price data and calculate log returns
data_files = {
    'stock_long': (portfolio_long_tickers, 'data/daily_stock_quotes.csv'),
    'stock_short': (portfolio_short_tickers, 'data/daily_stock_quotes.csv'),
    'benchmark_long': (benchmark_long, 'data/daily_benchmark_quotes.csv'),
    'benchmark_short': (benchmark_short, 'data/daily_benchmark_quotes.csv')
}

quotes, returns = {}, {}
for key, (tickers, file_path) in data_files.items():
    quotes[key] = load_and_filter_data(file_path, tickers, start_date, end_date)
    returns[key] = np.log(quotes[key] / quotes[key].shift(1)).dropna()

print(f"Data loaded: {len(quotes['stock_long'])} observations")

Found 5 of 5 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 2 of 2 tickers in data/daily_stock_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []
Found 1 of 1 tickers in data/daily_benchmark_quotes.csv
Missing tickers: []
Data loaded: 2515 observations


## **Train-Test Split and Portfolio Construction**

**Data Partitioning Strategy:**
- **Training Set (80%)**: Used for strategy development and parameter estimation
- **Test Set (20%)**: Provides unbiased out-of-sample performance validation
- **Portfolio Returns**: Calculated using normalized weights to ensure proper allocation
- **Temporal Alignment**: Ensures consistent time series alignment across all instruments

In [12]:
def calculate_portfolio_return(returns_data, weights):
    """Calculate normalized weighted portfolio returns"""
    filtered_weights = {k: v for k, v in weights.items() if k in returns_data.columns}
    total_weight = sum(filtered_weights.values())
    normalized_weights = {k: v/total_weight for k, v in filtered_weights.items()}
    return returns_data[list(normalized_weights.keys())].multiply(pd.Series(normalized_weights), axis=1).sum(axis=1)

# Calculate portfolio returns and perform train-test split
portfolio_long_return = calculate_portfolio_return(returns['stock_long'], weights_long)
portfolio_short_return = calculate_portfolio_return(returns['stock_short'], weights_short)

split_idx_long, split_idx_short = int(len(returns['stock_long']) * 0.8), int(len(returns['stock_short']) * 0.8)
test_set_long, test_set_short = quotes['stock_long'].iloc[split_idx_long:], quotes['stock_short'].iloc[split_idx_short:]

print(f"Train-Test Split: {split_idx_long}/{len(test_set_long)} (Long), {split_idx_short}/{len(test_set_short)} (Short)")
print(f"Test Period: {test_set_long.index[0].strftime('%Y-%m-%d')} to {test_set_long.index[-1].strftime('%Y-%m-%d')}")

Train-Test Split: 2011/504 (Long), 2011/504 (Short)
Test Period: 2023-06-05 to 2025-06-06


## **Backtesting - `bt`**

**Strategy Implementation using `bt` Library:**
- **Long-term Strategy**: Quarterly rebalancing (66 trading days) to minimize transaction costs and capture secular trends
- **Short-term Strategy**: Weekly rebalancing for tactical asset allocation and market timing
- **Risk Integration**: Incorporates risk-free rate for accurate risk-adjusted performance metrics
- **Benchmark Comparison**: Systematic evaluation against market indices with identical rebalancing frequencies
- **Transaction Cost Consideration**: Rebalancing frequencies optimized for strategy characteristics

In [13]:
def run_backtest(test_set, portfolio_tickers, benchmark_ticker, benchmark_quotes, weights, strategy_name, rebalance_freq):
    """Execute systematic backtest for portfolio vs benchmark"""
    all_quotes = test_set.copy()
    all_quotes[benchmark_ticker] = benchmark_quotes[benchmark_ticker].loc[test_set.index[0]:test_set.index[-1]]
    
    strategies = [
        bt.Strategy(f'{strategy_name} Portfolio', [rebalance_freq, bt.algos.SelectAll(), bt.algos.WeighSpecified(**weights), bt.algos.Rebalance()]),
        bt.Strategy(benchmark_ticker, [rebalance_freq, bt.algos.SelectThese([benchmark_ticker]), bt.algos.WeighEqually(), bt.algos.Rebalance()])
    ]
    
    backtests = [bt.Backtest(strategies[0], all_quotes[portfolio_tickers]), bt.Backtest(strategies[1], all_quotes[[benchmark_ticker]])]
    result = bt.run(*backtests)
    result.set_riskfree_rate(risk_free_rate)
    return result

# Execute backtests with appropriate rebalancing frequencies
print("Executing Backtests...")
result_long = run_backtest(test_set_long, portfolio_long_tickers, benchmark_long, quotes['benchmark_long'], weights_long, "Long-term", bt.algos.RunEveryNPeriods(66, offset=66))
result_short = run_backtest(test_set_short, portfolio_short_tickers, benchmark_short, quotes['benchmark_short'], weights_short, "Short-term", bt.algos.RunWeekly())

# Extract returns for QuantStats analysis
def extract_returns(result, strategy_name):
    return result[strategy_name].prices.pct_change().dropna()

bt_long_returns, bt_short_returns = extract_returns(result_long, 'Long-term Portfolio'), extract_returns(result_short, 'Short-term Portfolio')
bt_benchmark_long_returns, bt_benchmark_short_returns = extract_returns(result_long, benchmark_long), extract_returns(result_short, benchmark_short)

# Display backtest results
for result, title in [(result_long, 'Long-term'), (result_short, 'Short-term')]:
    print(f"\n{title} Backtest Results:")
    result.display()
    result.plot(figsize=(12, 6), title=f'{title} Portfolio vs Benchmark (Test Period)')

Executing Backtests...


100%|██████████| 2/2 [00:00<00:00,  6.49it/s]



Long-term Backtest Results:
Stat                 Long-term Portfolio    YYY
-------------------  ---------------------  ----------
Start                2023-06-04             2023-06-04
End                  2025-06-06             2025-06-06
Risk-free rate       4.24%                  4.24%

Total Return         30.66%                 21.52%
Daily Sharpe         0.68                   0.57
Daily Sortino        1.10                   0.82
CAGR                 14.26%                 10.20%
Max Drawdown         -13.82%                -14.44%
Calmar Ratio         1.03                   0.71

MTD                  2.03%                  0.70%
3m                   -1.23%                 0.52%
6m                   -3.84%                 -1.71%
YTD                  2.85%                  3.13%
1Y                   2.71%                  5.88%
3Y (ann.)            14.26%                 10.20%
5Y (ann.)            -                      -
10Y (ann.)           -                      -
Since Incep

## **HTML Reporting - `QuantStats`**

**Comprehensive QuantStats Reporting:**
- **Risk Metrics**: Value-at-Risk (VaR), Conditional VaR, Maximum Drawdown, Rolling Volatility
- **Return Analytics**: Total Return, CAGR, Best/Worst Periods, Win/Loss Ratios, Consistency Metrics
- **Risk-Adjusted Performance**: Sharpe Ratio, Sortino Ratio, Calmar Ratio, Information Ratio
- **Comparative Analysis**: Alpha/Beta decomposition, Tracking Error, Active Return Attribution
- **Statistical Tests**: Performance significance testing and confidence intervals
- **Visual Analytics**: Interactive charts, drawdown analysis, return distribution plots

In [14]:
def generate_reports(portfolio_returns, benchmark_returns, portfolio_name, benchmark_name, suffix):
    """Generate comprehensive QuantStats performance reports"""
    portfolio_returns.name = f"{portfolio_name} Portfolio"
    print(f"Generating {portfolio_name} reports ({portfolio_returns.index[0]} to {portfolio_returns.index[-1]})")
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        qs.reports.html(
            portfolio_returns, 
            benchmark_returns, 
            rf=risk_free_rate, 
            figsize=(8, 5),
            output=f'portfolios/portfolio_vs_benchmark_{suffix}-{datetime.date(end_date)}.html',
            title=f'{portfolio_name} Portfolio vs {benchmark_name}', 
            benchmark_title=benchmark_name,
            strategy_title=f'{portfolio_name} Portfolio' 
        )
        qs.reports.full(
            portfolio_returns, 
            benchmark_returns, 
            rf=risk_free_rate, 
            figsize=(8, 5),
            title=f'{portfolio_name} Portfolio vs {benchmark_name}', 
            benchmark_title=benchmark_name,
            strategy_title=f'{portfolio_name} Portfolio' 
        )
    return portfolio_returns, benchmark_returns

# Generate comprehensive performance reports
portfolio_long_test, benchmark_long_test = generate_reports(bt_long_returns, bt_benchmark_long_returns, "Long-term", benchmark_long, "long_term")
portfolio_short_test, benchmark_short_test = generate_reports(bt_short_returns, bt_benchmark_short_returns, "Short-term", benchmark_short, "short_term")

Generating Long-term reports (2023-06-05 00:00:00 to 2025-06-06 00:00:00)
Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_long_term-2025-06-06.html


                           YYY         Long-term Portfolio
-------------------------  ----------  ---------------------
Start Period               2023-09-11  2023-09-11
End Period                 2025-06-06  2025-06-06
Risk-Free Rate             4.24%       4.24%
Time in Market             91.0%       100.0%

Cumulative Return          21.52%      30.66%
CAGR﹪                     8.05%       11.22%

Sharpe                     0.67        0.77
Prob. Sharpe Ratio         66.99%      67.9%
Smart Sharpe               0.62        0.72
Sortino                    0.9         1.12
Smart Sortino              0.84        1.05
Sortino/√2                 0.63        0.79
Smart Sortino/√2           0.59        0.74
Omega                      1.15        1.15

Max Drawdown               -14.44%     -13.82%
Longest DD Days            106         186
Volatility (ann.)          11.64%      16.32%
R^2                        0.41        0.41
Information Ratio          0.02        0.02
Calmar            

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-12-03,2025-04-08,2025-06-06,186,-13.822469,-12.462337
2,2024-07-17,2024-08-05,2024-10-08,84,-9.134753,-8.451557
3,2023-09-15,2023-10-27,2023-11-09,56,-6.355421,-5.790792
4,2024-10-15,2024-10-31,2024-11-22,39,-4.725651,-4.603396
5,2024-03-28,2024-04-19,2024-05-09,43,-3.498312,-3.459600


Generating Short-term reports (2023-06-05 00:00:00 to 2025-06-06 00:00:00)
Added download button and removed QuantStats attribution from portfolios/portfolio_vs_benchmark_short_term-2025-06-06.html


                           XRLV        Short-term Portfolio
-------------------------  ----------  ----------------------
Start Period               2023-06-06  2023-06-06
End Period                 2025-06-06  2025-06-06
Risk-Free Rate             4.24%       4.24%
Time in Market             99.0%       100.0%

Cumulative Return          23.06%      30.17%
CAGR﹪                     7.42%       9.51%

Sharpe                     0.61        0.64
Prob. Sharpe Ratio         66.19%      62.56%
Smart Sharpe               0.6         0.64
Sortino                    0.85        0.93
Smart Sortino              0.84        0.93
Sortino/√2                 0.6         0.66
Smart Sortino/√2           0.6         0.65
Omega                      1.12        1.12

Max Drawdown               -9.6%       -12.69%
Longest DD Days            189         248
Volatility (ann.)          11.31%      16.15%
R^2                        0.19        0.19
Information Ratio          0.01        0.01
Calmar          

None

,Start,Valley,End,Days,Max Drawdown,99% Max Drawdown
1,2024-09-24,2025-04-22,2025-05-29,248,-12.694038,-11.946886
2,2023-07-26,2023-09-26,2023-10-12,79,-8.537142,-8.169293
3,2024-04-01,2024-04-18,2024-05-20,50,-4.674870,-4.199076
4,2024-05-23,2024-05-30,2024-07-03,42,-4.582566,-4.410285
5,2023-12-14,2023-12-20,2024-01-11,29,-3.385346,-3.328923


## **Performance Summary**

**Key Performance Indicators (Out-of-Sample Test Period):**

This section provides actionable insights for investment decision-making through:
- **Risk-Adjusted Returns**: Sharpe ratio comparison for risk-adjusted performance evaluation
- **Absolute Performance**: Total return analysis to assess wealth creation potential
- **Relative Performance**: Portfolio alpha generation versus benchmark indices
- **Statistical Significance**: Performance persistence and reliability assessment
- **Strategy Validation**: Out-of-sample results confirm strategy robustness and deployment readiness

In [15]:
def safe_metric(func, data, default=0.0):
    """Calculate metrics with error handling"""
    try:
        result = func(data)
        return result.iloc[0] if isinstance(result, pd.Series) else result
    except:
        return default

# Performance summary analysis
test_data = [('Long-term', portfolio_long_test, benchmark_long_test, benchmark_long),
             ('Short-term', portfolio_short_test, benchmark_short_test, benchmark_short)]

print("\n" + "="*60)
print("PORTFOLIO PERFORMANCE SUMMARY (OUT-OF-SAMPLE TEST PERIOD)")
print("="*60)

for strategy_name, portfolio_data, benchmark_data, benchmark_name in test_data:
    metrics = {
        'Portfolio Sharpe Ratio': safe_metric(qs.stats.sharpe, portfolio_data),
        'Benchmark Sharpe Ratio': safe_metric(qs.stats.sharpe, benchmark_data),
        'Portfolio Total Return': safe_metric(qs.stats.comp, portfolio_data),
        'Benchmark Total Return': safe_metric(qs.stats.comp, benchmark_data)
    }
    
    print(f"\n{strategy_name} Portfolio vs {benchmark_name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

print(f"\n📊 Reports: portfolios/portfolio_vs_benchmark_*-{datetime.date(end_date)}.html")
print(f"✅ Analysis Complete! Review HTML reports for detailed analytics.")


PORTFOLIO PERFORMANCE SUMMARY (OUT-OF-SAMPLE TEST PERIOD)

Long-term Portfolio vs YYY:
  Portfolio Sharpe Ratio: 0.9559
  Benchmark Sharpe Ratio: 0.9534
  Portfolio Total Return: 0.3066
  Benchmark Total Return: 0.2152

Short-term Portfolio vs XRLV:
  Portfolio Sharpe Ratio: 0.8974
  Benchmark Sharpe Ratio: 0.9749
  Portfolio Total Return: 0.3017
  Benchmark Total Return: 0.2306

📊 Reports: portfolios/portfolio_vs_benchmark_*-2025-06-06.html
✅ Analysis Complete! Review HTML reports for detailed analytics.
